# Tải dữ liệu cities và language-codes 
- Tải dữ liệu cities từ https://simplemaps.com/data/world-cities và upload lên MongoDB
- Tải dữ liệu language-codes từ https://datahub.io/core/language-codes và upload lên MongoDB


In [1]:
# Kiểm tra database và collection, tạo nếu chưa có
from pymongo import MongoClient
import pandas as pd

MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "smart_travel_v2"
COLLECTION_NAME = "world_cities"

client = MongoClient(MONGO_URI)

# Kiểm tra database đã tồn tại chưa
db_list = client.list_database_names()
if DB_NAME not in db_list:
    print(f"Database '{DB_NAME}' chưa tồn tại. Sẽ được tạo khi thêm dữ liệu đầu tiên.")
else:
    print(f"Database '{DB_NAME}' đã tồn tại.")

db = client[DB_NAME]

# Kiểm tra collection đã tồn tại chưa
col_list = db.list_collection_names()
if COLLECTION_NAME not in col_list:
    print(f"Collection '{COLLECTION_NAME}' chưa tồn tại. Sẽ được tạo khi thêm dữ liệu đầu tiên.")
else:
    print(f"Collection '{COLLECTION_NAME}' đã tồn tại.")

# Nếu muốn tạo collection thủ công (không cần thiết, MongoDB sẽ tự tạo khi insert)
if COLLECTION_NAME not in col_list:
    db.create_collection(COLLECTION_NAME)
    print(f"Đã tạo collection '{COLLECTION_NAME}'.")

# Lấy dữ liệu nếu collection đã có
collection = db[COLLECTION_NAME]
cities_cursor = collection.find({})
cities = list(cities_cursor)
print(f"Số lượng thành phố lấy được: {len(cities)}")
# Chuyển list cities thành DataFrame trước khi xử lý
if cities:
    df_cities = pd.DataFrame(cities)
    df_cities = df_cities[["city", "country", "lat", "lng", "iso2"]]
    df_cities["iso2"] = df_cities["iso2"].str.lower()
    display(df_cities.head())
else:
    print("Không có dữ liệu city nào trong collection.")

Database 'smart_travel_v2' đã tồn tại.
Collection 'world_cities' đã tồn tại.
Số lượng thành phố lấy được: 48059


,city,country,lat,lng,iso2
0,Tokyo,Japan,35.6870,139.7495,jp
1,Jakarta,Indonesia,-6.1750,106.8275,id
2,Delhi,India,28.6100,77.2300,in
3,Guangzhou,China,23.1300,113.2600,cn
4,Mumbai,India,19.0761,72.8775,in


# Lấy dữ liệu places từ API
https://rapidapi.com/gmapplatform/api/google-map-places-new-v2/playground/apiendpoint_ea6640da-90f7-4e89-8590-508da9401990

In [2]:
from pymongo import MongoClient
import requests, json, os, time

# Các loại địa điểm muốn tìm
PLACE_TYPES = ["tourist_attraction", "restaurant", "lodging"]

# Lưu dữ liệu vào MongoDB collection 'places'
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "smart_travel_v2"
COLLECTION_NAME = "places"

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
if COLLECTION_NAME not in db.list_collection_names():
    db.create_collection(COLLECTION_NAME)
    print(f"Đã tạo collection '{COLLECTION_NAME}'.")
collection = db[COLLECTION_NAME]

# API info
url = "https://google-map-places-new-v2.p.rapidapi.com/v1/places:searchText"

# headers = {
# 	"x-rapidapi-key": "cf1b379a98msh116f2d78aa3d55ep1a4602jsndbd91f1a8bb4",
# 	"x-rapidapi-host": "google-map-places-new-v2.p.rapidapi.com",
# 	"Content-Type": "application/json",
# 	"X-Goog-FieldMask": "*"
# }

# headers = {
# 	"x-rapidapi-key": "ffbaceaaeamsh9084aa32f4d5dfdp13028bjsn2366c1d9a5c9",
# 	"x-rapidapi-host": "google-map-places-new-v2.p.rapidapi.com",
# 	"Content-Type": "application/json",
# 	"X-Goog-FieldMask": "*"
# }

headers = {
	"x-rapidapi-key": "9036bbd933mshe2c1df67124cf82p17e447jsn62bbd79e7831",
	"x-rapidapi-host": "google-map-places-new-v2.p.rapidapi.com",
	"Content-Type": "application/json",
	"X-Goog-FieldMask": "*"
}

if 'df_cities' in locals():
    for row in df_cities[25:26].itertuples(index=False): # lan  sau chay tu 25: hochiminh
        print(f"\nXử lý thành phố: {row.city}, {row.country}")
        city = row.city
        country = row.country
        lat = row.lat
        lng = row.lng
        iso2 = row.iso2
        city_name = f"{city.lower()}({country.lower()})"

        for place_type in PLACE_TYPES:
            textQuery = place_type.replace("_", " ") + f" in {city_name}"
            print(f"\nĐang tìm {place_type} tại {city_name}...")

            payload = {
                "textQuery": textQuery,
                "languageCode": "en",
                "regionCode": iso2,
                "rankPreference": 0,
                "includedType": place_type,
                "openNow": True,
                "minRating": 0,
                "maxResultCount": 100,
                "priceLevels": [],
                "strictTypeFiltering": True,
                "locationBias": {
                    "circle": {
                        "center": {"latitude": lat, "longitude": lng},
                        "radius": 10000
                    }
                }
            }

            next_page_token = None
            while True:
                if next_page_token:
                    payload["pageToken"] = next_page_token
                try:
                    r = requests.post(url, json=payload, headers=headers)
                    print(f"  -> HTTP status: {r.status_code}")
                    resp = r.json()

                    # nếu không phải 200 thì in rõ lỗi trả về
                    if r.status_code != 200:
                        print("  -> Lỗi HTTP:", r.status_code, "| Nội dung:", resp)
                        # nếu hết quota/rate limit thì dừng luôn
                        if r.status_code == 429:
                            print("  -> Bị rate-limit/hết quota, dừng lại hoặc đổi API key.")
                            break
                        else:
                            break

                    results = resp.get("places") or resp.get("results") or []
                    next_page_token = resp.get("nextPageToken") or resp.get("next_page_token")

                    if results:
                        for place in results:
                            place["search_type"] = place_type
                            place["city"] = city
                            place["country"] = country
                        inserted = collection.insert_many(results)
                        print(f"  -> Đã lưu {len(inserted.inserted_ids)} kết quả vào MongoDB.")
                    else:
                        print("  -> Không có kết quả.")

                    if not next_page_token:
                        break  # Không còn trang tiếp theo
                    time.sleep(1)  # tránh rate limit giữa các lần gọi
                except Exception as e:
                    print(f"  -> Exception xảy ra: {e}")
                    break
else:
    print("Không tìm thấy DataFrame df_cities. Hãy khởi tạo df_cities trước.")



Xử lý thành phố: Istanbul, Turkey

Đang tìm tourist_attraction tại istanbul(turkey)...
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.

Đang tìm restaurant tại istanbul(turkey)...
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.

Đang tìm lodging tại istanbul(turkey)...
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.
  -> HTTP status: 200
  -> Đã lưu 20 kết quả vào MongoDB.


## Tiền xử lý dữ liệu

In [1]:
# Kiểm tra database và collection, tạo nếu chưa có
from pymongo import MongoClient
import pandas as pd

MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "smart_travel_v2"
COLLECTION_NAME = "places"

client = MongoClient(MONGO_URI)

# Kiểm tra database đã tồn tại chưa
db_list = client.list_database_names()
if DB_NAME not in db_list:
    print(f"Database '{DB_NAME}' chưa tồn tại. Sẽ được tạo khi thêm dữ liệu đầu tiên.")
else:
    print(f"Database '{DB_NAME}' đã tồn tại.")

db = client[DB_NAME]

# Kiểm tra collection đã tồn tại chưa
col_list = db.list_collection_names()
if COLLECTION_NAME not in col_list:
    print(f"Collection '{COLLECTION_NAME}' chưa tồn tại. Sẽ được tạo khi thêm dữ liệu đầu tiên.")
else:
    print(f"Collection '{COLLECTION_NAME}' đã tồn tại.")

# Lấy dữ liệu nếu collection đã có
collection = db[COLLECTION_NAME]
places_cursor = collection.find({})
places = list(places_cursor)


Database 'smart_travel_v2' đã tồn tại.
Collection 'places' đã tồn tại.


In [8]:
import json

print(json.dumps(places[200], indent=2, ensure_ascii=False, default=str))

{
  "_id": "68c823f908482f98a7661446",
  "name": "places/ChIJ76KhEr-rNTERXYzy4QH-Xnk",
  "id": "ChIJ76KhEr-rNTERXYzy4QH-Xnk",
  "types": [
    "jewelry_store",
    "finance",
    "store",
    "point_of_interest",
    "establishment"
  ],
  "nationalPhoneNumber": "024 3826 7117",
  "internationalPhoneNumber": "+84 24 3826 7117",
  "formattedAddress": "130 P. Hàng Bạc, Hàng Buồm, Hoàn Kiếm, Hà Nội, Vietnam",
  "addressComponents": [
    {
      "longText": "130",
      "shortText": "130",
      "types": [
        "street_number"
      ],
      "languageCode": "en"
    },
    {
      "longText": "Phố Hàng Bạc",
      "shortText": "P. Hàng Bạc",
      "types": [
        "route"
      ],
      "languageCode": "vi"
    },
    {
      "longText": "Phố cổ Hà Nội",
      "shortText": "Phố cổ Hà Nội",
      "types": [
        "neighborhood",
        "political"
      ],
      "languageCode": "vi"
    },
    {
      "longText": "Hoàn Kiếm",
      "shortText": "Hoàn Kiếm",
      "types": [
       

In [12]:
from typing import Dict, List, Optional
import re
from datetime import datetime

def preprocess_place(p: dict) -> dict:
    """
    Preprocess a place dictionary from Google Places API.
    Args:
        p (dict): Raw place data from the API.
    Returns:
        dict: Processed place data with cleaned and standardized fields.
    """
    processed = {}

    # 1. Basic Identification
    processed["id"] = p.get("id", "").strip()
    if not processed["id"]:
        raise ValueError("Missing required field: id")

    # 2. Name and Display Name
    processed["name"] = p.get("name", "").strip()
    display_name = p.get("displayName", {})
    processed["display_name"] = display_name.get("text", processed["name"]).strip()
    processed["language_code"] = display_name.get("languageCode", "en").strip()

    # 3. Types
    processed["types"] = p.get("types", [])
    processed["primary_type"] = p.get("primaryType", processed["types"][0] if processed["types"] else "").strip()
    primary_type_display = p.get("primaryTypeDisplayName", {})
    processed["primary_type_display_name"] = primary_type_display.get("text", processed["primary_type"]).strip()

    # 4. Phone Numbers
    processed["phone_number"] = p.get("internationalPhoneNumber", p.get("nationalPhoneNumber", "")).strip()
    if processed["phone_number"]:
        # Remove non-digit characters except '+' for international format
        processed["phone_number"] = re.sub(r"[^\d+]", "", processed["phone_number"])

    # 5. Address
    processed["formatted_address"] = p.get("formattedAddress", "").strip()
    processed["short_formatted_address"] = p.get("shortFormattedAddress", processed["formatted_address"]).strip()

    # 6. Location
    location = p.get("location", {})
    processed["latitude"] = float(location.get("latitude", 0))
    processed["longitude"] = float(location.get("longitude", 0))
    # Validate coordinates
    if not (-90 <= processed["latitude"] <= 90 and -180 <= processed["longitude"] <= 180):
        processed["latitude"], processed["longitude"] = 0, 0  # Default to invalid coordinates

    # 7. Viewport (optional, simplified)
    viewport = p.get("viewport", {})
    processed["viewport"] = {
        "low": {
            "latitude": float(viewport.get("low", {}).get("latitude", 0)),
            "longitude": float(viewport.get("low", {}).get("longitude", 0))
        },
        "high": {
            "latitude": float(viewport.get("high", {}).get("latitude", 0)),
            "longitude": float(viewport.get("high", {}).get("longitude", 0))
        }
    }

    # 8. Rating and Reviews
    processed["rating"] = float(p.get("rating", 0))
    processed["user_rating_count"] = int(p.get("userRatingCount", 0))
    reviews = p.get("reviews", [])
    processed["reviews"] = [
        {
            "name": r.get("name", "").strip(),
            "rating": float(r.get("rating", 0)),
            "publish_time": r.get("publishTime", ""),
            "relative_publish_time": r.get("relativePublishTimeDescription", "").strip()
        } for r in reviews # [:3]  # Limit to 3 reviews for brevity
    ]

    # 9. URLs
    processed["google_maps_uri"] = p.get("googleMapsUri", "").strip()
    processed["website_uri"] = p.get("websiteUri", "").strip()
    # Validate URLs
    for url_field in ["google_maps_uri", "website_uri"]:
        if processed[url_field] and not processed[url_field].startswith("http"):
            processed[url_field] = f"https://{processed[url_field]}"

    # 10. Opening Hours
    regular_hours = p.get("regularOpeningHours", {})
    processed["open_now"] = bool(regular_hours.get("openNow", False))
    processed["weekday_descriptions"] = regular_hours.get("weekdayDescriptions", [])
    processed["opening_hours"] = [
        {
            "open": {
                "day": period.get("open", {}).get("day", 0),
                "hour": period.get("open", {}).get("hour", 0),
                "minute": period.get("open", {}).get("minute", 0)
            },
            "close": {
                "day": period.get("close", {}).get("day", 0),
                "hour": period.get("close", {}).get("hour", 0),
                "minute": period.get("close", {}).get("minute", 0)
            }
        } for period in regular_hours.get("periods", [])
    ]

    # 11. Business Status
    processed["business_status"] = p.get("businessStatus", 0)  # Assume 0 = OPERATIONAL

    # 12. Services (boolean flags)
    services = [
        "takeout", "delivery", "dineIn", "curbsidePickup", "reservable",
        "servesBreakfast", "servesLunch", "servesDinner", "servesBeer",
        "servesWine", "servesBrunch", "servesVegetarianFood",
        "outdoorSeating", "liveMusic", "menuForChildren", "servesCocktails",
        "servesDessert", "servesCoffee", "goodForChildren", "allowsDogs",
        "restroom", "goodForGroups", "goodForWatchingSports"
    ]
    for service in services:
        processed[service] = bool(p.get(service, False))

    # 13. Payment Options
    payment_options = p.get("paymentOptions", {})
    processed["payment_options"] = {
        "accepts_credit_cards": bool(payment_options.get("acceptsCreditCards", False)),
        "accepts_debit_cards": bool(payment_options.get("acceptsDebitCards", False)),
        "accepts_cash_only": bool(payment_options.get("acceptsCashOnly", False)),
        "accepts_nfc": bool(payment_options.get("acceptsNfc", False))
    }

    # 14. Parking Options
    parking_options = p.get("parkingOptions", {})
    processed["parking_options"] = {
        "free_parking_lot": bool(parking_options.get("freeParkingLot", False)),
        "paid_parking_lot": bool(parking_options.get("paidParkingLot", False)),
        "free_street_parking": bool(parking_options.get("freeStreetParking", False)),
        "paid_street_parking": bool(parking_options.get("paidStreetParking", False)),
        "valet_parking": bool(parking_options.get("valetParking", False)),
        "free_garage_parking": bool(parking_options.get("freeGarageParking", False)),
        "paid_garage_parking": bool(parking_options.get("paidGarageParking", False))
    }

    # 15. Accessibility Options
    accessibility_options = p.get("accessibilityOptions", {})
    processed["accessibility_options"] = {
        "wheelchair_accessible_parking": bool(accessibility_options.get("wheelchairAccessibleParking", False)),
        "wheelchair_accessible_entrance": bool(accessibility_options.get("wheelchairAccessibleEntrance", False)),
        "wheelchair_accessible_restroom": bool(accessibility_options.get("wheelchairAccessibleRestroom", False)),
        "wheelchair_accessible_seating": bool(accessibility_options.get("wheelchairAccessibleSeating", False))
    }

    # 16. Photos (limit to first photo for simplicity)
    photos = p.get("photos", [])
    processed["photos"] = [
        {
            "name": photo.get("name", "").strip(),
            "author": photo.get("authorAttributions", [{}])[0].get("displayName", "").strip()
        } for photo in photos # [:1]  # Limit to 1 photo
    ]

    # 17. Editorial Summary
    editorial_summary = p.get("editorialSummary", {})
    processed["editorial_summary"] = editorial_summary.get("text", "").strip()

    # 18. Price Level
    processed["price_level"] = int(p.get("priceLevel", 0))

    return processed

print(json.dumps(preprocess_place(places[200]), indent=2, ensure_ascii=False, default=str))

{
  "id": "ChIJ76KhEr-rNTERXYzy4QH-Xnk",
  "name": "places/ChIJ76KhEr-rNTERXYzy4QH-Xnk",
  "display_name": "Quang Huy Gemstone",
  "language_code": "en",
  "types": [
    "jewelry_store",
    "finance",
    "store",
    "point_of_interest",
    "establishment"
  ],
  "primary_type": "jewelry_store",
  "primary_type_display_name": "Jewelry store",
  "phone_number": "+842438267117",
  "formatted_address": "130 P. Hàng Bạc, Hàng Buồm, Hoàn Kiếm, Hà Nội, Vietnam",
  "short_formatted_address": "130 P. Hàng Bạc, Phố cổ Hà Nội, Hàng Bạc",
  "latitude": 21.0341679,
  "longitude": 105.850923,
  "viewport": {
    "low": {
      "latitude": 21.0327839697085,
      "longitude": 105.84957636970849
    },
    "high": {
      "latitude": 21.035481930291507,
      "longitude": 105.8522743302915
    }
  },
  "rating": 4.7,
  "user_rating_count": 683,
  "reviews": [
    {
      "name": "places/ChIJ76KhEr-rNTERXYzy4QH-Xnk/reviews/Ci9DQUlRQUNvZENodHljRjlvT2tJNVEwNVdZM2hKVG5kMWJHSnNjakp2UmpCcVVsRRAB",
    

In [13]:
import json
from bson import ObjectId

# Lọc tất cả các đối tượng có primaryType là 'tourist_attraction'
tourist_attractions = [p for p in places if p.get("primaryType") == "tourist_attraction"]
print(f"Số lượng tourist_attraction: {len(tourist_attractions)}")
# # In ra 1 ví dụ
# if tourist_attractions:
#     print(json.dumps(tourist_attractions[0], indent=2, ensure_ascii=False, default=str))
# else:
#     print("Không tìm thấy đối tượng nào.")


Số lượng tourist_attraction: 435


# Tính khoảng cách dùng API
https://rapidapi.com/sa307485/api/real-time-distance-calculator-with-map

In [ ]:
import requests

url = "https://real-time-distance-calculator-with-map.p.rapidapi.com/rapidapi/distance-calculator/calculate.php"

querystring = {"endLongitude":"2.3522","startLongitude":"-0.1278","startLatitude":"51.5074","endLatitude":"48.8566"}

headers = {
	"x-rapidapi-key": "ffbaceaaeamsh9084aa32f4d5dfdp13028bjsn2366c1d9a5c9",
	"x-rapidapi-host": "real-time-distance-calculator-with-map.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(json.dumps(response.json(), indent=2, ensure_ascii=False, default=str))

{
  "status": "success",
  "startLatitude": "51.5074",
  "startLongitude": "-0.1278",
  "startLocation": "Charing Cross",
  "endLatitude": "48.8566",
  "endLongitude": "2.3522",
  "endLocation": "Place de l'Hôtel de Ville",
  "distance": {
    "kilometers": 343.56,
    "miles": 213.48
  },
  "travel_times": {
    "plane": 0.38,
    "train": 2.86,
    "car": 4.29,
    "bike": 17.18,
    "walk": 68.71
  },
  "google_maps_link": "https://www.google.com/maps/dir/?api=1&origin=51.5074,-0.1278&destination=48.8566,2.3522",
  "message": "Distance and travel time calculation successful."
}


In [6]:
import requests
import json  # cần import để dùng json.dumps

url = "https://real-time-distance-calculator-with-map.p.rapidapi.com/rapidapi/distance-calculator/calculate.php"

# Thay bằng tọa độ Hà Nội và TP.HCM
querystring = {
    "startLatitude": "21.0285",   # Hà Nội
    "startLongitude": "105.8542",
    "endLatitude": "10.7626",     # TP.HCM
    "endLongitude": "106.6602"
}

headers = {
    "x-rapidapi-key": "ffbaceaaeamsh9084aa32f4d5dfdp13028bjsn2366c1d9a5c9",
    "x-rapidapi-host": "real-time-distance-calculator-with-map.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# In kết quả với định dạng đẹp + hỗ trợ tiếng Việt
print(json.dumps(response.json(), indent=2, ensure_ascii=False, default=str))


{
  "status": "success",
  "startLatitude": "21.0285",
  "startLongitude": "105.8542",
  "startLocation": "Phố Đinh Tiên Hoàng",
  "endLatitude": "10.7626",
  "endLongitude": "106.6602",
  "endLocation": "Hẻm 189 Đường Lý Thường Kiệt",
  "distance": {
    "kilometers": 1144.75,
    "miles": 711.33
  },
  "travel_times": {
    "plane": 1.27,
    "train": 9.54,
    "car": 14.31,
    "bike": 57.24,
    "walk": 228.95
  },
  "google_maps_link": "https://www.google.com/maps/dir/?api=1&origin=21.0285,105.8542&destination=10.7626,106.6602",
  "message": "Distance and travel time calculation successful."
}


In [7]:
import requests
import json

url = "https://real-time-distance-calculator-with-map.p.rapidapi.com/rapidapi/distance-calculator/calculate.php"

# ✅ Đổi sang tọa độ Hồ Tây -> Nhà Hát Lớn Hà Nội
querystring = {
    "startLatitude": "21.0578",   # Hồ Tây
    "startLongitude": "105.8180",
    "endLatitude": "21.0247",     # Nhà Hát Lớn
    "endLongitude": "105.8575"
}

headers = {
    "x-rapidapi-key": "ffbaceaaeamsh9084aa32f4d5dfdp13028bjsn2366c1d9a5c9",
    "x-rapidapi-host": "real-time-distance-calculator-with-map.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(json.dumps(response.json(), indent=2, ensure_ascii=False, default=str))


{
  "status": "success",
  "startLatitude": "21.0578",
  "startLongitude": "105.8180",
  "startLocation": "Phố Quảng Khánh",
  "endLatitude": "21.0247",
  "endLongitude": "105.8575",
  "endLocation": "Phố Tràng Tiền",
  "distance": {
    "kilometers": 5.51,
    "miles": 3.42
  },
  "travel_times": {
    "plane": 0.01,
    "train": 0.05,
    "car": 0.07,
    "bike": 0.28,
    "walk": 1.1
  },
  "google_maps_link": "https://www.google.com/maps/dir/?api=1&origin=21.0578,105.8180&destination=21.0247,105.8575",
  "message": "Distance and travel time calculation successful."
}


In [1]:
import redis

r = redis.Redis(host="localhost", port=6379, decode_responses=True)
print(r.ping())  # True nếu kết nối OK
r.set("test_key", "hello redis")
print(r.get("test_key"))


True
hello redis
